In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled","true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize","1073741824")

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 3, Finished, Available, Finished)

In [2]:
from pyspark.sql.functions import monotonically_increasing_id

#Create DimCustomer
df_dimCustomer = spark.sql("select sc.* , sca.AddressID, sca.AddressType from silver.salesCustomer sc join silver.salesCustomerAddress sca on sc.customerid = sca.customerid")

#Add surrogate key as the first column
df_dimCustomer_with_surrogate_key = df_dimCustomer.withColumn("CustomerIDKey", monotonically_increasing_id())\
    .select(
        "CustomerIDKey",  # Select the surrogate key column first
        *[column for column in df_dimCustomer.columns if column != "CustomerIDKey"]  # Select the remaining columns in their original order
    )

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 4, Finished, Available, Finished)

In [3]:
display(df_dimCustomer_with_surrogate_key)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4c2ea6aa-fcc2-4717-8829-461e18e23162)

In [4]:
from pyspark.sql.functions import monotonically_increasing_id

#Create dimProduct
df_dimProduct = <ENTER HERE>



#Add surrogate key as the first column
df_dimProduct_with_surrogate_key = df_dimProduct.withColumn("ProductIDKey", monotonically_increasing_id())\
    .select(
        "ProductIDKey",  # Select the surrogate key column first
        *[column for column in df_dimProduct.columns if column != "ProductIDKey" and column!="spc.Name"]  # Select the remaining columns in their original order
    )

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 6, Finished, Available, Finished)

In [5]:
display(df_dimProduct_with_surrogate_key)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4034949b-0593-47d5-9d9c-5499dc37f3f7)

In [6]:
from pyspark.sql.functions import expr
 
# Define the start and end dates for your DimDate table
start_date = "2000-01-01"
end_date = "2024-12-31"
 
# Create a DataFrame with a range of dates
df_dimDate = spark.range(0, (spark.sql("SELECT datediff('{0}', '{1}')".format(end_date, start_date)).collect()[0][0])+1) \
    .selectExpr("CAST(id AS INT) AS id") \
    .selectExpr("date_add('{0}', id) AS Date".format(start_date))
 
# Extract different date components
df_dimDate = df_dimDate \
    .withColumn("Year", expr("year(Date)")) \
    .withColumn("Month", expr("month(Date)")) \
    .withColumn("DayOfMonth", expr("dayofmonth(Date)")) \
    .withColumn("DayOfYear", expr("dayofyear(Date)")) \
    .withColumn("WeekOfYear", expr("weekofyear(Date)")) \
    .withColumn("DayOfWeek", expr("dayofweek(Date)")) \
    .withColumn("Quarter", expr("quarter(Date)"))

display(df_dimDate)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0697a8bc-b576-418d-9bdf-11094468b1d6)

# Write dataframes to Gold Layer as External Tables

In [7]:
# Write dimCustomer table into Gold Lakehouse as Delta Table
basePathGoldLakeHouse = "<ENTER HERE>"
tableName="dimCustomer"
df_dimCustomer_with_surrogate_key.write.mode("overwrite").format("delta").save(basePathGoldLakeHouse + '//' + tableName)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 9, Finished, Available, Finished)

In [8]:
# Write dimProduct table into Gold Lakehouse as Delta Table
tableName="dimProduct"
df_dimProduct_with_surrogate_key.write.mode("overwrite").format("delta").save(basePathGoldLakeHouse + '//' +tableName)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 10, Finished, Available, Finished)

In [9]:
#Create factSales
df_factSales = <ENTER HERE>
display(df_factSales)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 54706cbf-5e16-4b5f-adc7-9e9e2434413e)

In [10]:
# Write dimDate table into Gold Lakehouse as Delta Table

tableName="dimDate"
df_dimDate.write.mode("overwrite").format("delta").save(basePathGoldLakeHouse + '//' + tableName)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 12, Finished, Available, Finished)

In [11]:
# Write factSales table into Gold Lakehouse as Delta Table

tableName="factSales"
df_factSales.write.mode("overwrite").format("delta").save(basePathGoldLakeHouse + '//' + tableName)

StatementMeta(, bc2e4292-277a-4546-8fd2-ebaec0604566, 13, Finished, Available, Finished)